# 🔧 Environment Setup

In [1]:
!pip install -U numpy==1.26.4 pandas scikit-learn     langchain langchain-openai langchain-community     qdrant-client neo4j


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.3/62.3 kB 549.9 kB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 91.2/91.2 kB 1.6 MB/s eta 0:00:00ta 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 68.0/68.0 kB 710.5 kB/s eta 0:00:00 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.9/73.9 kB 1.2 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.9/41.9 kB 416.8 kB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.5/40.5 kB 635.9 kB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.2/14.2 MB 15.9 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.8/11.8 MB 22.1 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.2/12.2 MB 25.2 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 14.7 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 70.4/70.4 kB 8.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━

## 🛠 Fix numpy bool compatibility (temporary workaround)

In [2]:
import numpy as np
if not hasattr(np, 'bool'):
    np.bool = bool


/tmp/ipykernel_122/2245577656.py:2: FutureWarning: In the future `np.bool` will be defined as the corresponding NumPy scalar.
  if not hasattr(np, 'bool'):


## 📁 Load Telecom Data

In [3]:
import pandas as pd

templates_df = pd.read_csv('templates.csv')
projects_df = pd.read_csv('projects_tasks.csv')
templates_df.head(), projects_df.head()


(    template_id          template_name  default_duration_days  task_count  \
 0  TEMPLATE-001  5G Tower Installation                      2           5   
 1  TEMPLATE-002        Fiber Trenching                     13           3   
 2  TEMPLATE-003   Site Acceptance Test                      2           3   
 3  TEMPLATE-004        Fiber Trenching                     10           3   
 4  TEMPLATE-005        RF Optimization                      8           4   
 
                 required_roles  
 0         Technician, Engineer  
 1    Engineer, Project Manager  
 2                 Engineer, QA  
 3  Technician, Project Manager  
 4          QA, Project Manager  ,
    project_id      task_id   template_id                          task_name  \
 0  PROJ-00001  TASK-000001  TEMPLATE-009        Power Backup Setup - Survey   
 1  PROJ-00001  TASK-000002  TEMPLATE-009    Power Backup Setup - Excavation   
 2  PROJ-00001  TASK-000003  TEMPLATE-009  Power Backup Setup - Cable Laying   
 3  P

## 🧠 Neo4j Graph Ingestion

In [4]:
from neo4j import GraphDatabase

uri = "bolt://graph-neo4j:7687"
auth = ("neo4j", "password")
driver = GraphDatabase.driver(uri, auth=auth)
driver.verify_connectivity()
print("✅ Neo4j connection successful!")

cypher_query = '''
MERGE (p:Project {id: $project_id})
MERGE (t:Task {id: $task_id, name: $task_name, status: $status, risk: $risk})
MERGE (tpl:Template {id: $template_id})
MERGE (p)-[:HAS_TASK]->(t)
MERGE (t)-[:USES_TEMPLATE]->(tpl)
'''

def insert_row(tx, row):
    tx.run(cypher_query, 
           project_id=row['project_id'],
           task_id=row['task_id'],
           task_name=row['task_name'],
           status=row['status'],
           risk=row['risk'],
           template_id=row['template_id'])

with driver.session() as session:
    for i, row in projects_df.iterrows():
        session.write_transaction(insert_row, row)
        if i % 1000 == 0:
            print(f"Inserted {i} records...")


✅ Neo4j connection successful!


/tmp/ipykernel_122/3709484692.py:28: DeprecationWarning: write_transaction has been renamed to execute_write
  session.write_transaction(insert_row, row)


Inserted 0 records...
Inserted 1000 records...


## 📦 Vector Store Ingestion to Qdrant

In [10]:
from qdrant_client import QdrantClient
from langchain_community.vectorstores import Qdrant
from langchain_openai import OpenAIEmbeddings

# Monkey patch fix for NumPy
import numpy as np
if not hasattr(np, 'bool'):
    np.bool = bool

# Embeddings (uses env var OPENAI_API_KEY)
embedding_model = OpenAIEmbeddings()

# Connect to Qdrant container by name (inside Docker)
qdrant_client = QdrantClient(
    host="vector-qdrant", port=6333
)

# Convert your data
texts = [f"{row.template_name} - Roles: {row.required_roles}" for _, row in templates_df.iterrows()]
metadata = templates_df.to_dict(orient='records')

# Correct way to build vector store
vectorstore = Qdrant.from_texts(
    texts=texts,
    embedding=embedding_model,
    metadatas=metadata,
    collection_name="telecom_templates",
    url="http://vector-qdrant:6333"  # 👈 Instead of `qdrant_client=...`
)


## 🧠 Agent with Graph + Vector RAG

In [11]:
from langchain.agents import initialize_agent, Tool
from langchain.chat_models import ChatOpenAI
from langchain.chains.graph_qa.cypher import GraphCypherQAChain
from langchain.chains import RetrievalQA
from langchain.graphs import Neo4jGraph

llm = ChatOpenAI(temperature=0, model_name="gpt-4")
graph = Neo4jGraph(url=uri, username="neo4j", password="password")

cypher_chain = GraphCypherQAChain.from_llm(
    llm, graph=graph, verbose=True, allow_dangerous_requests=True
)

retriever_chain = RetrievalQA.from_chain_type(
    llm=llm, retriever=vectorstore.as_retriever(), verbose=True
)

tools = [
    Tool(name="Graph QA", func=cypher_chain.run, description="Use for schema and task relationships"),
    Tool(name="Template Lookup", func=retriever_chain.run, description="Use for template descriptions")
]

agent = initialize_agent(tools, llm, agent="zero-shot-react-description", verbose=True)


/tmp/ipykernel_122/188872435.py:7: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import ChatOpenAI``.
  llm = ChatOpenAI(temperature=0, model_name="gpt-4")
/tmp/ipykernel_122/188872435.py:8: LangChainDeprecationWarning: The class `Neo4jGraph` was deprecated in LangChain 0.3.8 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-neo4j package and should be used instead. To use it run `pip install -U :class:`~langchain-neo4j` and import as `from :class:`~langchain_neo4j import Neo4jGraph``.
  graph = Neo4jGraph(url=uri, username="neo4j", password="password")
/tmp/ipykernel_122/188872435.py:23: LangChainDeprecationWarning: LangChain agents will continue to be supported, but 

## 🔍 Ask Questions to Agent

In [14]:
agent.run("What is the description of TEMPLATE-009?")




> Entering new AgentExecutor chain...
I need to find the description of a specific template. The Template Lookup tool should be able to provide this information.
Action: Template Lookup
Action Input: {"template_id": "TEMPLATE-009"}

> Entering new RetrievalQA chain...

> Finished chain.

Observation: I'm sorry, but you didn't ask a question. Could you please provide more information?
Thought:The Template Lookup tool didn't provide the expected result. It seems like it didn't understand the input. I should try again with a different format.
Action: Template Lookup
Action Input: {"id": "TEMPLATE-009"}

> Entering new RetrievalQA chain...

> Finished chain.

Observation: I'm sorry, but your message doesn't contain a question. Could you please provide more details or ask a specific question?
Thought:The Template Lookup tool still didn't provide the expected result. It seems like it's not understanding the input. I should try again with a different format.
Action: Template Lookup
Action I

'The description of TEMPLATE-009, also known as the Site Acceptance Test, is a type of testing performed by a QA (Quality Assurance) professional and an Engineer. The roles involved in this process are the QA and the Engineer. The QA is responsible for ensuring the quality of the product or system, while the Engineer is responsible for the technical aspects of the product or system.'